# Notebook to convert a Pytroch model to a HEF file

In [2]:
# import the ClientRunner class from the hailo_sdk_client package
from hailo_sdk_client import ClientRunner
import onnx
import sys
import os
import numpy as np
import torch
import clip
import open_clip
from pathlib import Path
from onnxsim import simplify
from PIL import Image

# Own modules
print(Path.cwd())
sys.path.append(str(Path.cwd().parent))
os.chdir(str(Path.cwd().parent))
import pathsToFolders as ptf #Controlls all paths #

/home/lukasschoepf/Documents/ProjWork1_DFC/python


Paths and names for models

In [36]:
chosen_hw_arch = "hailo8l"

har_path = ptf.HarPath
hef_path = ptf.Hefpath


## Get ONNX file

Diffrent for CLIP / TinyCLIP


For CLIP

In [14]:
model_name = "RN50x4"

device = "cuda" if torch.cuda.is_available() else "cpu" 
model, preprocess = clip.load(model_name, device=device)

vit = model.visual
input_pixels = model.visual.input_resolution
input_image = torch.rand(1, 3, input_pixels, input_pixels)

input_image = preprocess(Image.open("hailoDFC/pics/car-967387_1280.png")).unsqueeze(0).to(device)
onnx_path = str(ptf.onnxFolder/ f"{model_name}.onnx")
print(f"Input: {input_pixels}")
vit.eval()
torch.onnx.export(vit,
            input_image,
            onnx_path,
            opset_version=14,
            do_constant_folding=True,)

# Simplify the ONNX model to resolve mismatches
simplified_model, check = simplify(onnx_path)
model_path_simple = f"hailoDFC/models/{model_name}_simple.onnx"

# Save the simplified model
onnx.save(simplified_model, model_path_simple)

print(f"Simple Model saved at {model_path_simple}")

Input: 288
Simple Model saved at hailoDFC/models/RN50x4_simple.onnx


For TinyCLIP


In [45]:
model_name = "TinyCLIP-ResNet-19M-Text-19M" # TinyCLIP-ResNet-19M-Text-19M or TinyCLIP-ResNet-30M-Text-29M

device = "cuda" if torch.cuda.is_available() else "cpu" 
model, *_ = open_clip.create_model_and_transforms(model_name,pretrained=str(ptf.tinyClipModels / f"{model_name}-LAION400M.pt"))

vit = model.visual 
input_pixels = model.visual.image_size

print(f"Input: {input_pixels}")
input_image = torch.rand(1, 3, input_pixels, input_pixels)
# input_image = preprocess(Image.open("hailoDFC/pics/car-967387_1280.png")).unsqueeze(0).to(device)

onnx_path = str(ptf.onnxFolder/ f"{model_name}.onnx")
vit.eval()
torch.onnx.export(vit,
         input_image,
         onnx_path,
         opset_version=14,
         do_constant_folding=True,)

# Simplify the ONNX model to resolve mismatches
simplified_model, check = simplify(onnx_path)
model_name = "TinyRN"
model_path_simple = f"hailoDFC/models/{model_name}_simple.onnx"

# Save the simplified model
onnx.save(simplified_model, model_path_simple)

print(f"Simple Model saved at {model_path_simple}")

/home/lukasschoepf/Documents/ProjWork1_DFC/src/open_clip/factory.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=m

Input: 224
Simple Model saved at hailoDFC/models/TinyRN_simple.onnx


Use model from Hailo

In [46]:
model_path_simple = "hailoDFC/models/baseAndSimple/TinyRN_simple.onnx"
model_name = "tinyCLIP_RN50"

### Onnx to Har

Changes to onnx_graph.py

```python
def get_spatial_unflatten_reshape_info(self):
        if self.op == "Reshape": # if self.op != "Reshape":
```

In [47]:
runner = ClientRunner(hw_arch=chosen_hw_arch)


print(f"Take model from {model_path_simple}")
hn, npz = runner.translate_onnx_model(
    model_path_simple,
    model_name
)

hailo_model_har_name = str(har_path / f"{model_name}_hailo_model.har")
runner.save_har(hailo_model_har_name)

Take model from hailoDFC/models/baseAndSimple/TinyRN_simple.onnx
[info] Translation started on ONNX model tinyCLIP_RN50
[info] Restored ONNX model tinyCLIP_RN50 (completion time: 00:00:00.10)
[info] Extracted ONNXRuntime meta-data for Hailo model (completion time: 00:00:00.33)
[info] Simplified ONNX model for a parsing retry attempt (completion time: 00:00:00.65)


IndexError: list index out of range

## Optimize Har file

The next step is to optimze the har file. To do this we need a calibration dataset.

In [39]:
# preprocessing
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize, PILToTensor
import os

try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC


datafolder = ptf.dataBaseFolder
input_folder = ptf.Dataset5Patch
calibFolder = datafolder / "calibData"

Functions to preprocess the input images to the right resolution

In [40]:
def _convert_image_to_rgb(image):
    return image.convert("RGB")

def transform(n_px):
    """
    n_px: input resolution of the network
    """
    return Compose([
        Resize(n_px, interpolation=BICUBIC),
        CenterCrop(n_px),
        _convert_image_to_rgb,
        ToTensor(),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

`x_y_pixel` should be the input resolution.

In [41]:
x_y_pixel = 224
preprocess = transform(x_y_pixel)
images_list = [img_name for img_name in os.listdir(input_folder) if os.path.splitext(img_name)[1] == ".jpg"]
images_list = images_list[0:1024]
calib_dataset = np.zeros((len(images_list), x_y_pixel, x_y_pixel, 3))

for idx, img_name in enumerate(sorted(images_list)):
    img = Image.open(os.path.join(input_folder, img_name))
    # img = PILToTensor(img)
    img_preproc = preprocess(img)
    img_transposed = np.transpose(img_preproc.numpy(),(1,2,0))
    input_data = (img_transposed * 255).astype(np.uint8)  # Assuming image is already normalized
    calib_dataset[idx, :, :, :] = img_transposed

np.save(calibFolder / f"calib_set_{model_name}.npy", calib_dataset)

We will load our parsed HAR from the Parsing Tutorial

In [42]:
# Second, we will load our parsed HAR from the Parsing Tutorial

hailo_model_har_name = f"{model_name}_hailo_model.har"
hailo_model_har_path = har_path / hailo_model_har_name
assert os.path.isfile(hailo_model_har_path), "Please provide valid path for HAR file"
runner = ClientRunner(har=str(hailo_model_har_path),hw_arch=chosen_hw_arch)

In [43]:
# Batch size is 8 by default
# alls = "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n" # From tutorial

# Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)), # From Lia
#alls = "normalization1 = normalization([0.48145466, 0.4578275, 0.40821073], [0.26862954, 0.26130258, 0.27577711])\n"

# Load the model script to ClientRunner so it will be considered on optimization
# runner.load_model_script(alls)

# Call Optimize to perform the optimization process
runner.optimize(calib_dataset)

# Save the result state to a Quantized HAR file
quantized_model_har_path = str(har_path / f"{model_name}_quantized_model.har")
runner.save_har(quantized_model_har_path)
print(f"saved model at {quantized_model_har_path}")

[info] Starting Model Optimization
[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[warning] Running model optimization with zero level of optimization is not recommended for production use and might lead to suboptimal accuracy results
[info] Model received quantization params from the hn
[info] Starting Mixed Precision
[info] Mixed Precision is done (completion time is 00:00:00.01)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration


Calibration: 100%|██████████| 64/64 [00:00<00:00, 85.09entries/s] 

[info] Statistics Collector is done (completion time is 00:00:00.78)
[info] Starting Fix zp_comp Encoding
[info] Fix zp_comp Encoding is done (completion time is 00:00:00.00)
[info] Matmul Equalization skipped


[info] Finetune encoding skipped
[info] Bias Correction skipped
[info] Adaround skipped
[info] Quantization-Aware Fine-Tuning skipped
[info] Layer Noise Analysis skipped
[info] The calibration set seems to not be normalized, because the values range is [(-1.7922626, 1.9303361), (-1.7520971, 2.0748837), (-1.4802198, 2.145897)].
Since the neural core works in 8-bit (between 0 to 255), a quantization will occur on the CPU of the runtime platform.
Add a normalization layer to the model to offload the normalization to the neural core.
Refer to the user guide Hailo Dataflow Compiler user guide / Model Optimization / Optimization Related Model Script Commands / model_modification_commands / normalization for details.
[info] Model Optimization is done
[info] Saved HAR to: /home/lukasschoepf/Documents/ProjWork1_DFC/hailoDFC/models/Harfiles/superSmall_CLIP_RN50_quantized_model.har
saved model at hailoDFC/models/Harfiles/superSmall_CLIP_RN50_quantized_model.har


## Compile the quantized model

Now we can compile the quantized model

In [44]:
quantized_model_har_path = str(har_path / f"{model_name}_quantized_model.har")
print(f"Model used:{model_name}_quantized_model.har")
runner = ClientRunner(har=quantized_model_har_path,hw_arch=chosen_hw_arch)
# By default it uses the hw_arch that is saved on the HAR. It is not recommended to change the hw_arch after Optimization.

hef = runner.compile()

file_name = str(hef_path / f"{model_name}.hef")
with open(file_name, "wb") as f:
    f.write(hef)

har_path =ptf.HarPath/ f"{model_name}_compiled_model.har"
runner.save_har(har_path)

Model used:superSmall_CLIP_RN50_quantized_model.har
[info] To achieve optimal performance, set the compiler_optimization_level to "max" by adding performance_param(compiler_optimization_level=max) to the model script. Note that this may increase compilation time.
[info] Loading network parameters
[info] Starting Hailo allocation and compilation flow
[info] Using Single-context flow
[info] Resources optimization guidelines: Strategy -> GREEDY Objective -> MAX_FPS
[info] Resources optimization params: max_control_utilization=75%, max_compute_utilization=75%, max_compute_16bit_utilization=75%, max_memory_utilization (weights)=75%, max_input_aligner_utilization=75%, max_apu_utilization=75%
[info] Using Single-context flow
[info] Resources optimization guidelines: Strategy -> GREEDY Objective -> MAX_FPS
[info] Resources optimization params: max_control_utilization=75%, max_compute_utilization=75%, max_compute_16bit_utilization=75%, max_memory_utilization (weights)=75%, max_input_aligner_uti